In [1]:
from datasets import Dataset, DatasetDict
import json
import pandas as pd
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [26]:
# Open the JSONL file
train_list =[]
with open('train.jsonl', 'r') as file:
	for line in file:
		# Parse each line as JSON
		json_obj = json.loads(line)
		
		# Process the JSON object (example: print it)
		# print(json_obj)
		train_list.append(json_obj['text'])

# print('train_list', train_list)
print('train_list[0]', train_list[0:2])
print(len(train_list))
# data = DatasetDict({'train':Dataset.from_dict({"intstruction":train_list}), 'test':Dataset.from_dict({"intstruction":train_list[0:1]})})
data = Dataset.from_dict({"intstruction":train_list})
print(data)

train_list[0] ['<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n"阿猛，如果你要cos一个角色，你会选择谁呢？"<|im_end|>\n<|im_start|>assistant\n阿猛可以cos大嫂吗 或者缸灿团队来cos个狂飙想看！！[星星眼][星星眼][星星眼]<|im_end|>', '<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n"怎样让中年男同事瞬间变得时尚硬汉？"<|im_end|>\n<|im_start|>assistant\n建议把小缸改造成卡车硬汉：李有田[doge]<|im_end|>']
15318
Dataset({
    features: ['intstruction'],
    num_rows: 15318
})


In [9]:
print(data.column_names)

['intstruction']


In [10]:
model_path= '/root/autodl-tmp/qwen/Qwen2-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
print(tokenizer.pad_token_id)
print(tokenizer.decode(tokenizer.pad_token_id))
print(tokenizer.pad_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


151643
<|endoftext|>
<|endoftext|>


In [27]:
def tokenize_function(examples):
    # extract text
    text = examples["intstruction"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        f"{text}",
        # return_tensors="np", # return numpy array
        add_special_tokens=False
    )
    # print(text)
    # print(tokenized_inputs)
    # return {
    #     "input_ids": tokenized_inputs['input_ids'] + [tokenizer.pad_token_id],
    #     "attention_mask": tokenized_inputs['attention_mask'] + [1],
    #      "labels": tokenized_inputs['input_ids'] + [tokenizer.pad_token_id]
    # }
    return {
        "input_ids": tokenized_inputs['input_ids'] ,
        "attention_mask": tokenized_inputs['attention_mask'] ,
         "labels": tokenized_inputs['input_ids']
    }

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, remove_columns=data.column_names)
print(tokenized_data)
print(tokenized_data[0])
tokenizer.decode(tokenized_data[0]['input_ids'])

Map:   0%|          | 0/15318 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 15318
})
{'input_ids': [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 151645, 198, 151644, 872, 198, 1, 99727, 100525, 3837, 102056, 30534, 9407, 46944, 100780, 3837, 102762, 50404, 100165, 101036, 11319, 1, 151645, 198, 151644, 77091, 198, 99727, 100525, 73670, 9407, 26288, 109790, 101037, 92313, 28946, 107347, 103322, 103932, 36407, 9407, 18947, 101231, 103794, 99172, 50930, 31251, 58, 109014, 99246, 1457, 109014, 99246, 1457, 109014, 99246, 60, 151645], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 151645, 198, 151644, 872, 198, 1, 99727, 100525, 3837, 102056, 30534, 9407, 46944, 100780, 3837, 102762, 50404, 100165, 101036, 11319, 1, 151645, 198, 151644, 77091, 198, 99727, 10

'<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n"阿猛，如果你要cos一个角色，你会选择谁呢？"<|im_end|>\n<|im_start|>assistant\n阿猛可以cos大嫂吗 或者缸灿团队来cos个狂飙想看！！[星星眼][星星眼][星星眼]<|im_end|>'

In [13]:
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto",torch_dtype=torch.bfloat16)
model.enable_input_require_grads()
model.dtype

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

torch.bfloat16

In [14]:
# lora config
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
print(config)


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)


In [15]:
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.26434798934534914


In [28]:
# 配置训练参数
args = TrainingArguments(
    output_dir="/root/loraoutput/Qwen2_instruct_lora2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=50, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)
print('done!')
print(args)

done!
TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.NO,
evaluation_strategy=None,
fp16=False,


In [29]:
print(tokenizer.decode(tokenized_data[0]['input_ids']))
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_data,
    # eval_dataset=tokenized_data["test"],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
"阿猛，如果你要cos一个角色，你会选择谁呢？"<|im_end|>
<|im_start|>assistant
阿猛可以cos大嫂吗 或者缸灿团队来cos个狂飙想看！！[星星眼][星星眼][星星眼]<|im_end|>


In [30]:
trainer.train()

/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.920400
20,2.713000
30,2.765600
40,2.622200
50,2.633000
60,2.681000
70,2.601200
80,2.577800
90,2.506700
100,2.554800


/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.p

RuntimeError: [enforce fail at inline_container.cc:595] . unexpected pos 85132544 vs 85132432

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel
mode_path = '/root/autodl-tmp/qwen/Qwen2-7B-Instruct/'
lora_path = '/root/loraoutput/Qwen2_instruct_lora2/checkpoint-2000' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "你是谁？"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)